In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 46kB/s 


In [ ]:
!nvidia-smi

Mon Nov  2 11:32:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DIR_ROOT = '/content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc'
DIR_INPUT_TRAIN = DIR_ROOT + '/InputTrain_30_10_2020_2'
DIR_INPUT_TEST = DIR_ROOT + '/Tests/test'
DIR_MODEL_LSTM = DIR_ROOT + '/Models/LSTM_Train_30_10_2020_2.h5'
DIR_MODEL_CNN = DIR_ROOT + '/Models/VGG16_Model.h5'
SIZE = (224, 224)
NUM_FRAME_INPUT_LSTM = 20
TRANSFER_VALUE_SIZE = 4096
RNN_SIZE = 512
EPOCH = 300
BATCH_SIZE = 150

VIDEO_NAMES = [
  'da',
  'dn',
  'nt',
  'no'
]

VIDEO_NAMES_DETAIL = [
  'Da',
  'Danh, tat',
  'Nam toc',
  'Binh thuong'
]

VIDEO_LABELS = [
  [1, 0, 0, 0],
  [0, 1, 0, 0],
  [0, 0, 1, 0],
  [0, 0, 0, 1]
]

NUM_CLASSIFY = len(VIDEO_NAMES)

In [ ]:
from keras.models import load_model
from keras.models import Model
from random import shuffle
import numpy as np

In [ ]:
import os
import cv2
import sys
import zipfile

def fun_print(name: str, value) -> None:
    print('@ Deep Learning> ', name)
    print(value)


def fun_getFileNames(path: str) -> list:
    return os.listdir(path)


def fun_showVideoPath(path: str, delay: int = 25) -> None:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    while True:
        if not isContinue:
            break
        cv2.imshow('frame', frame)
        if cv2.waitKey(delay=delay) & 0xFF == ord('q'):
            break
        isContinue, frame = cap.read()

    cap.release()
    cv2.destroyAllWindows()


def fun_getFramesOfVideo(path: str, count: int = 20) -> list:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    imgs = []
    while count > 0:
        if not isContinue:
            break
        imgs.append(frame)
        isContinue, frame = cap.read()
        count -= 1
    cap.release()
    cv2.destroyAllWindows()
    return imgs

def fun_getFramesOfVideo_ALL(path: str) -> list:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    imgs = []
    while True:
        if not isContinue:
            break
        imgs.append(frame)
        isContinue, frame = cap.read()
    cap.release()
    cv2.destroyAllWindows()
    return imgs


def fun_showVideoFrames(frames: list, delay: int = 25) -> None:
    for frame in frames:
        cv2.imshow('frame', frame)
        if cv2.waitKey(delay=delay) & 0xFF == ord('q'):
            break


def fun_showVideo(source, delay: int = 25) -> None:
    if isinstance(source, str):
        fun_showVideoPath(path=source, delay=delay)
    else:
        fun_showVideoFrames(frames=source, delay=delay)


def fun_resizeFrames(frames: list, size: tuple = (224, 224)) -> list:
    imgs = []
    count = 0
    for frame in frames:
        try:
          fr = cv2.resize(frame, dsize=size)
        except:
          print('\r!Error To Resize of {0}'.format(count))
        imgs.append(fr)
        count += 1
    cv2.destroyAllWindows()
    return imgs


def fun_saveFramesToVideo(frames: list, path: str, fps: int = 25) -> bool:
    try:
        height, width, layer = frames[0].shape
        wr = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
        for frame in frames:
            wr.write(frame)
        wr.release()
        cv2.destroyAllWindows()
        return True

    except:
        fun_print(name='Write Video: '+path, value='ERROR TO WRITE VIDEO')
        return False

def fun_getSizeOfFrame(frame) -> tuple:
    height, width, layer = frame.shape
    return (width, height)

# version 1
def fun_outListVideoWithNumFrame(pathVideoLoad: str, dirToSave: str, preFixName: str, videoNameIndex: int= None, countFrame: int = 40, fps: int = 25, isShowCalculating: bool = False) -> int:
    if videoNameIndex is None:
        fun_print('fun_outListVideoWithNumFrame', 'Please input para: videoNameIndex')
        return 0

    all = 0
    countWriten = 0
    if isShowCalculating:
        fun_print('Calculator Video Out Frame', 'calculating...')
        all = fun_getFramesOfVideo_ALL(pathVideoLoad)
        all = len(all) // countFrame

    cap = cv2.VideoCapture(pathVideoLoad)
    isContinue, frame = cap.read()
    count = videoNameIndex
    while True:
        if not isContinue:
            break
        nameFile = dirToSave + preFixName + '_out_'+str(count)+'.avi'
        cFrame = countFrame
        frames = []

        # get list frame
        while cFrame > 0:
            frames.append(frame)
            isContinue, frame = cap.read()
            if not isContinue:
                break
            cFrame -= 1

        # check video enough frameCount frame ?
        if len(frames) != countFrame:
            break

        # write list frame
        res = fun_saveFramesToVideo(frames=frames, path=nameFile, fps=fps)
        countWriten += 1
        if res:
            if isShowCalculating:
                percent = countWriten / all
                mess = '\r - Writen: {0} -> Complete: {1:.1%}'.format(nameFile, percent)
                sys.stdout.write(mess)
                sys.stdout.flush()
            else:
                mess = '\r - Writen: {0} -> Complete'.format(nameFile)
                sys.stdout.write(mess)
                sys.stdout.flush()

        # done
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return count

def fun_extractZipFile(pathFileZip: str, pathToSave: str) -> None:
    if not os.path.exists(pathToSave):
        os.makedirs(pathToSave)
    fun_print(name='Extract All ' + pathFileZip, value='Extracting...')
    if (pathFileZip.endswith('.zip')):
        zipfile.ZipFile(file=pathFileZip, mode='r').extractall(path=pathToSave)
        print('Extract Done')
    else:
        print('Please Input zip file')

def fun_print_process(count: int, max: int, mess: str = 'Processing: ') -> None:
  process = count / max
  mess = '\r - ' +  mess + ' [{0:.1%}]'.format(process)
  sys.stdout.write(mess)
  sys.stdout.flush()

def getModelLSTM(rnn_size: int = 512, input_shape: tuple = (20, 4096), num_classify: int = 3):
  modelLSTM = Sequential()
  modelLSTM.add(LSTM(rnn_size, input_shape= input_shape))
  modelLSTM.add(Dense(1024))
  modelLSTM.add(Activation('relu'))
  modelLSTM.add(Dense(50))
  modelLSTM.add(Activation('sigmoid'))
  modelLSTM.add(Dense(num_classify))
  modelLSTM.add(Activation('softmax'))
  modelLSTM.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
  return modelLSTM

def fun_predict(modelLSTM, transferValue, isPrint: bool= True):
    arrPre = []
    arrPre.append(transferValue)
    Real = modelLSTM.predict(np.array(arrPre))
    pre = np.argmax(Real)

    if isPrint:
      print(Real, pre)
      print('\r')
    return pre, Real[0][pre]

#version 1
def fun_loadVideoPredictPutTextAndSave(modelLSTM, pathLoadVideo, pathSave, videoNameDetail: list, perFameInputLSTM= 20, tranferSize: int = 4096, fps: int= 25):
    cap = cv2.VideoCapture(pathLoadVideo)
    isContinue, frame = cap.read()

    size = fun_getSizeOfFrame(frame= frame)
    wr = cv2.VideoWriter(pathSave, cv2.VideoWriter_fourcc(*'MJPG'), fps, size)
    countFrame = 0

    while True:
      if not isContinue:
        break
      
      imgs = []
      trans = []
      count = perFameInputLSTM

      while count > 0:
        imgs.append(frame)
        trans.append(frame)
        isContinue, frame = cap.read()
        count -= 1

      if len(imgs) != perFameInputLSTM:
        break

      trans = getTransferValue(trans)

      pre, real = fun_predict(modelLSTM= modelLSTM, transferValue= trans)

      color=(100, 200, 255)
      conv = videoNameDetail[pre]

      text = 'Predict: {0} -> Real: [ {1} ]'.format(conv, real)

      for ff in imgs:
        countFrame += 1
        # putText
        cv2.putText(img=ff,
                    text=text,
                    org=(50, 100),
                    fontScale=1,
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    thickness=1,
                    color= color )

        cv2.putText(img=ff,
                    text='Frame Count: {0}'.format(countFrame),
                    org=(50, 150),
                    fontScale=1,
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    thickness=1,
                    color= color )

        wr.write(ff)

      isContinue, frame = cap.read()

    wr.release()
    cap.release()
    cv2.destroyAllWindows()

def fun_mergeVideo(dirInput: str, videoNames: list, pathSave, fps: int= 25):
  count = 0
  max = len(videoNames)
  sizeVideoOut = (int(1280 * 0.7), int(720 * 0.7))
  wr = cv2.VideoWriter(pathSave, cv2.VideoWriter_fourcc(*'MJPG'), fps, sizeVideoOut)
  for name in videoNames:
    frames = fun_getFramesOfVideo_ALL(path= dirInput + name)
    frames = fun_resizeFrames(frames= frames, size= sizeVideoOut)
    for frame in frames:
      wr.write(frame)
    fun_print_process(count= count, max= max, mess= 'Merge video processing: ')
    count += 1
  wr.release()
  cv2.destroyAllWindows()

In [ ]:
fileNames = fun_getFileNames(path= DIR_INPUT_TEST)
fun_print(name= 'Xem thư mục bao nhiêu file?: ' + DIR_INPUT_TEST, value= 'Có: {0} Files'.format(len(fileNames)))

@ Deep Learning>  Xem thư mục bao nhiêu file?: /content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc/Tests/test
Có: 4 Files


In [ ]:
modelVGG16 = load_model(filepath= DIR_MODEL_CNN, compile= False)
modelVGG16.summary()
modelLSTM = load_model(filepath= DIR_MODEL_LSTM)
modelLSTM.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
transferLayer = modelVGG16.get_layer(name='fc2')
modelVGG16 = Model(inputs= modelVGG16.input, outputs= transferLayer.output)

In [ ]:
def getVideoLabelNames_EachFolder(path: str):
  names = []
  labels = []
  
  for fol in VIDEO_NAMES:
    folder = path + '/' + fol
    fileNames = fun_getFileNames(path= folder)
    index = VIDEO_NAMES.index(fol)
    for file in fileNames:
      names.append('/' + fol + '/' + file)
      labels.append(VIDEO_LABELS[index])
  
  c = list(zip(names, labels))
  shuffle(c)

  names, labels = zip(*c)
  return names, labels

In [ ]:
names, labels = getVideoLabelNames_EachFolder(path= DIR_INPUT_TEST)
fun_print('Names', names[0:5])
fun_print('labels', labels[0:5])

@ Deep Learning>  Names
('/dn/dn1_113_003.avi', '/no/no_002.avi', '/nt/fi_nt_004.avi', '/nt/fi_nt_001.avi', '/da/da1_111_003.avi')
@ Deep Learning>  labels
([0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0])


In [ ]:
def getTransferValue(pathVideoOrListFrame):
  if isinstance(pathVideoOrListFrame, str):
    frames = fun_getFramesOfVideo(path=pathVideoOrListFrame, count=NUM_FRAME_INPUT_LSTM)
  else:
    frames = pathVideoOrListFrame
    
  frames = fun_resizeFrames(frames= frames, size=SIZE)

  frames = np.array(frames)
  frames = (frames / 255.).astype(np.float16)

  transfer = modelVGG16.predict(frames)
  return transfer

In [ ]:
transfer = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TEST + '/da/da1_111_001.avi')
print(transfer)

[[0.         0.         1.6838499  ... 0.         0.         0.68654674]
 [0.         0.         1.6627777  ... 0.         0.         0.71237314]
 [0.         0.         1.6429672  ... 0.         0.         0.67726964]
 ...
 [0.         0.         1.5355805  ... 0.         0.         0.84070814]
 [0.         0.         1.5388265  ... 0.         0.         0.8411636 ]
 [0.         0.         1.4494164  ... 0.         0.         0.8075534 ]]


In [ ]:
def onesHotLabel(label: list):
  _ones = np.ones([NUM_FRAME_INPUT_LSTM, NUM_CLASSIFY])
  _onesHot = label * _ones
  return np.array(_onesHot)

In [ ]:
_oneHot = onesHotLabel([1, 0, 0, 0])
fun_print(name='oneHost', value= _oneHot)

@ Deep Learning>  oneHost
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [ ]:
names, labels = getVideoLabelNames_EachFolder(path= DIR_INPUT_TEST)

fun_print(name= 'Size of List video', value= len(names))
fun_print(name= 'Size of List labels', value= len(labels))

print('= '*50)

print(names[0:5])
print(labels[0:5])

print('= '*50)

print(getTransferValue(DIR_INPUT_TRAIN + names[0]))
print(onesHotLabel(labels[0]))

@ Deep Learning>  Size of List video
20
@ Deep Learning>  Size of List labels
20
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
('/da/da1_113_001.avi', '/nt/fi_nt_005.avi', '/nt/fi_nt_002.avi', '/nt/fi_nt_003.avi', '/no/no_001.avi')
([1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1])
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
[[0.         0.         1.6406014  ... 0.         0.         0.6860293 ]
 [0.         0.         1.671472   ... 0.         0.         0.6824765 ]
 [0.         0.         1.7053887  ... 0.         0.         0.7028643 ]
 ...
 [0.         0.         1.514013   ... 0.         0.         0.92988265]
 [0.         0.         1.5307286  ... 0.         0.         0.93455213]
 [0.         0.         1.5738035  ... 0.         0.         0.91121125]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0.

In [ ]:
def getTrainSet_LabelSet(numItem: int):
  count = 0
  trainSet = []
  labelSet = []
  while count < numItem:
    itemTrain = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TEST + names[count])
    itemLable = onesHotLabel(label= labels[count])

    trainSet.append(itemTrain)
    labelSet.append(itemLable[0])

    fun_print_process(count= count, max= numItem, mess= 'Video frame throw into VGG16 Model Processing: ')

    count += 1
  
  return trainSet, labelSet

In [ ]:
testSet, testLabelSet = getTrainSet_LabelSet(numItem= len(names))
fun_print(name= 'Get transfer value', value= 'Finish')
print('LEN TRAINT: ', len(testSet))
print('NUM FREAMS INPUT: ', len(testSet[0]))
print('SIZE TRANSFER: ', len(testSet[0][0]))

 - Video frame throw into VGG16 Model Processing:  [95.0%]@ Deep Learning>  Get transfer value
Finish
LEN TRAINT:  20
NUM FREAMS INPUT:  20
SIZE TRANSFER:  4096


In [ ]:
result = modelLSTM.evaluate(np.array(testSet), np.array(testLabelSet))
for name, value in zip(modelLSTM.metrics_names, result):
  print(name, value)

1/1 [==============================] - 0s 1ms/step - loss: 0.0366 - accuracy: 0.9000
loss 0.036638252437114716
accuracy 0.8999999761581421


In [ ]:
count = 1
for name in names:
  transfer = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TEST + name)
  pre, real = fun_predict(modelLSTM= modelLSTM, transferValue= transfer, isPrint= False)
  fun_print(name= 'predict video: ' + str(count)+ name, value= VIDEO_NAMES_DETAIL[pre])
  count += 1

@ Deep Learning>  predict video: 1/da/da1_113_001.avi
Da
@ Deep Learning>  predict video: 2/nt/fi_nt_005.avi
Nam toc
@ Deep Learning>  predict video: 3/nt/fi_nt_002.avi
Nam toc
@ Deep Learning>  predict video: 4/nt/fi_nt_003.avi
Nam toc
@ Deep Learning>  predict video: 5/no/no_001.avi
Binh thuong
@ Deep Learning>  predict video: 6/no/no_003.avi
Binh thuong
@ Deep Learning>  predict video: 7/no/no_005.avi
Binh thuong
@ Deep Learning>  predict video: 8/nt/fi_nt_004.avi
Nam toc
@ Deep Learning>  predict video: 9/no/no_002.avi
Binh thuong
@ Deep Learning>  predict video: 10/no/no_004.avi
Binh thuong
@ Deep Learning>  predict video: 11/dn/dn1_112_001.avi
Danh, tat
@ Deep Learning>  predict video: 12/nt/fi_nt_001.avi
Nam toc
@ Deep Learning>  predict video: 13/da/da1_111_003.avi
Danh, tat
@ Deep Learning>  predict video: 14/dn/dn1_113_001.avi
Danh, tat
@ Deep Learning>  predict video: 15/da/da1_111_002.avi
Da
@ Deep Learning>  predict video: 16/dn/dn1_112_002.avi
Danh, tat
@ Deep Learning>  

In [ ]:
fun_loadVideoPredictPutTextAndSave(modelLSTM= modelLSTM, pathLoadVideo= DIR_ROOT + '/Tests/video_test7.mp4', pathSave= DIR_ROOT + '/Tests/video_test7_out_2_11_2020.avi', videoNameDetail= VIDEO_NAMES_DETAIL)

[[0.9924664  0.00121097 0.00208111 0.0042416 ]] 0

[[0.9871034  0.00130305 0.00657069 0.00502283]] 0

[[2.6706156e-01 5.7396817e-04 6.9782847e-01 3.4536004e-02]] 2

[[7.8150797e-01 6.5447146e-04 1.8900414e-01 2.8833453e-02]] 0

[[0.991198   0.00149295 0.00229249 0.00501648]] 0

[[0.01180492 0.00107616 0.98220223 0.00491679]] 2

[[0.01997434 0.00246015 0.9720729  0.00549262]] 2

[[0.55067027 0.00243569 0.42927757 0.01761642]] 0

[[0.98845834 0.00156837 0.00322536 0.00674795]] 0

[[0.6114996  0.00426345 0.0045195  0.37971747]] 0

[[0.97916585 0.00236071 0.01271186 0.0057616 ]] 0

[[0.954458   0.02236244 0.01171085 0.01146872]] 0

[[4.9535930e-03 1.7106038e-04 7.6021630e-01 2.3465903e-01]] 2

[[2.7034909e-03 3.1386683e-04 9.9134773e-01 5.6349193e-03]] 2

[[1.3381468e-02 2.3069573e-04 9.4545978e-01 4.0928081e-02]] 2

[[2.2580829e-02 2.0925669e-04 7.9692495e-01 1.8028501e-01]] 2

[[8.3823912e-03 1.4410650e-04 4.2612991e-01 5.6534356e-01]] 3

[[9.8159024e-03 2.2470311e-04 9.5434105e-01 3.561

In [ ]:
videoMerge = fun_mergeVideo(dirInput= DIR_INPUT_TRAIN, videoNames= names, pathSave= DIR_ROOT + '/Tests/videoMerge.avi')

 - Merge video processing:  [95.0%]